# GIS-Project GEO885: Spatial Accessibility
*Authors: Alessandro Joshua Pierro 20-733-861, Fabio Schilling 19-704-048*     
*Date: 10.06.2024* 

## Load necessary libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import os

## Load data
### ZuriAct Data

In [2]:
# Define the Swiss CRS
crs_sui = 2056

# Read the ZüriACT data in & convert it to the Swiss CRS
zuriact = gpd.read_file('data/züriact_data.json')
zuriact = zuriact.to_crs(crs_sui)

In [3]:
# Drop all unecessary columns
zuriact = zuriact.drop(columns=['attribute_id', 'street_edge_id', 'osm_street_id', 'gsv_panorama_id', 'heading', 'pitch', 'zoom', 'canvas_x', 'canvas_y', 
                                'canvas_height', 'canvas_width', 'gsv_url', 'image_capture_date', 'label_date', 'notsure_count', 'label_description', 'user_id'])
# Drop the occlusion labels
zuriact = zuriact.drop(zuriact[zuriact['label_type'] == 'Occlusion'].index)

In [4]:
print(zuriact)

          label_type  neighborhood  severity  is_temporary  label_id  \
0     SurfaceProblem  Grossmünster       2.0         False       284   
1     SurfaceProblem      Oberdorf       3.0         False       924   
2           CurbRamp     Stadthaus       1.0         False      1854   
3           CurbRamp     Stadthaus       1.0         False      1855   
4           CurbRamp     Stadthaus       2.0         False      1853   
...              ...           ...       ...           ...       ...   
4912          Signal     Sihlporte       NaN         False      2758   
4913          Signal     Sihlporte       NaN         False      2762   
4914          Signal     Sihlporte       NaN         False      2764   
4915          Signal     Sihlporte       NaN         False      2795   
4916        CurbRamp  Bahnhofplatz       2.0         False      8193   

      label_severity  label_is_temporary  agree_count  disagree_count  \
0                3.0               False            2         

### Public transport stops

In [5]:
# Load the stops
stops_kreis1 = gpd.read_file('data/stops_kreis1.geojson')
stops_kreis1 = stops_kreis1.drop(columns = ['CNAME', 'index_right', 'bezeichnung'])

In [6]:
print(stops_kreis1)

    STOPID                        CHSTNAME                 VTYP  \
0        1          Zürich, Bahnhof Selnau                 Tram   
1        2          Zürich, Bahnhofquai/HB      Tram,Trolleybus   
2        3        Zürich Bürkliplatz (See)               Schiff   
3        4      Zürich, Rudolf-Brun-Brücke             Bus,Tram   
4        5        Zürich, Central Polybahn        Standseilbahn   
5        6                   Zürich HB SZU               S-Bahn   
6        7               Zürich Limmatquai               Schiff   
7        8                Zürich, Neumarkt  Bus,Tram,Trolleybus   
8        9  Zürich, ETH/Universitätsspital             Bus,Tram   
9       10           Zürich, Kantonsschule                 Tram   
10      11         Zürich, Bahnhofplatz/HB  Bus,Tram,Trolleybus   
11      12                 Zürich Storchen               Schiff   
12      13                 Zürich, Rathaus             Bus,Tram   
13      14            Zürich, Kantonalbank             Bus,Tra

## Analysis
### Looping through all stops to assign labels
This is where the fun starts

In [24]:
def process_stops_with_multiple_buffers(stops_kreis1, zuriact, output_directory, buffer_sizes=[25, 50, 100]):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Define weights and positive impact labels
    weights = {
        'SurfaceProblem': 3.0,
        'Obstacle': 5.0,
        'NoSidewalk': 3.0,
        'Other': 0.5,
        'NoCurbRamp': 4.0,
        'CurbRamp': -5.0,  # Negative because a CurbRamp is generally positive
        'Crosswalk': -2,  # Negative because it generally indicates a positive feature
        'Signal': -2      # Same as Crosswalk
    }
    positive_features = ['CurbRamp', 'Crosswalk', 'Signal']

    for buffer_size in buffer_sizes:
        # Create subdirectory for each buffer size
        buffer_output_directory = os.path.join(output_directory, f'buffer_{buffer_size}m')
        os.makedirs(buffer_output_directory, exist_ok=True)

        summary_data = []
        points_with_weights = []

        for index, stop in stops_kreis1.iterrows():
            # Buffer for each stop with current buffer size
            buffered_stop = stop.geometry.buffer(buffer_size)
            buffered_stop_gdf = gpd.GeoDataFrame(geometry=[buffered_stop], crs=stops_kreis1.crs)

            # Perform spatial join with the zuriact data for the specific buffer
            buffered_labels = gpd.sjoin(zuriact, buffered_stop_gdf, how='inner', predicate='intersects')

            # Handle lists and other non-primitive types in buffered_labels
            for column in buffered_labels.columns:
                if buffered_labels[column].apply(lambda x: isinstance(x, list)).any():
                    buffered_labels[column] = buffered_labels[column].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)

            # Adjust severity based on whether the feature is generally considered positive
            buffered_labels['adjusted_severity'] = buffered_labels.apply(
                lambda x: (6 - x['severity']) if x['label_type'] in positive_features else x['severity'],
                axis=1)

            # Calculate weighted severity for each obstacle within the buffer
            weighted_severity = buffered_labels['label_type'].map(weights) * buffered_labels['adjusted_severity']
            total_weighted_severity = weighted_severity.sum()
            count_points = len(buffered_labels)

            # Normalize the total weighted severity by the count of points
            normalized_severity = total_weighted_severity / count_points if count_points > 0 else 0

            # Save to individual GeoPackage file for each stop
            stop_id = stop['STOPID']
            chstname = stop['CHSTNAME']
            output_path = os.path.join(buffer_output_directory, f'buffered_{buffer_size}m_labels_stop_{stop_id}.gpkg')
            buffered_labels.to_file(output_path, layer=f'stop_{stop_id}', driver='GPKG')

            # Collect data for the summary and points with weighted severity
            summary_data.append({
                'STOPID': stop_id,
                'CHSTNAME': chstname,
                'geometry': buffered_stop,
                'average_weighted_severity': normalized_severity,
                'count_points': count_points
            })
            points_with_weights.append({
                'STOPID': stop_id,
                'CHSTNAME': chstname,
                'geometry': stop.geometry,
                'weighted_severity': normalized_severity,
                'count_points': count_points
            })

        # Create and save summary layer
        summary_gdf = gpd.GeoDataFrame(summary_data, crs=stops_kreis1.crs)
        summary_gdf.set_geometry('geometry', inplace=True)
        summary_gdf.to_file(os.path.join(buffer_output_directory, f'summary_stops_{buffer_size}m_weighted_severity.gpkg'), layer='summary_severity', driver='GPKG')

        # Create and save points layer
        points_gdf = gpd.GeoDataFrame(points_with_weights, crs=stops_kreis1.crs)
        points_gdf.set_geometry('geometry', inplace=True)
        points_gdf.to_file(os.path.join(buffer_output_directory, f'weighted_severity_points_{buffer_size}m.gpkg'), layer='weighted_severity_points', driver='GPKG')

# Example usage:
process_stops_with_multiple_buffers(stops_kreis1, zuriact, 'output_directory')

c:\Users\Fabio\anaconda3\envs\geo876\Lib\site-packages\geopandas\io\file.py:633: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
c:\Users\Fabio\anaconda3\envs\geo876\Lib\site-packages\geopandas\io\file.py:633: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
c:\Users\Fabio\anaconda3\envs\geo876\Lib\site-packages\geopandas\io\file.py:633: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
c:\Users\Fabio\anaconda3\envs\geo876\Lib\site-packages\geopandas\io\file.py:633: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, fil